In [1]:
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
train = pd.read_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\데이터정리중\재무변수_한국은행_Altman.csv',index_col=0)
# train = train.drop(['감사의견코드','공시제목','년'], axis=1)
train

dataset = pd.read_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\kind_bankrupt_Final.csv', index_col = 0)
print(train.shape, dataset.shape)

(27357, 21) (938, 6)


In [3]:
train.loc[(train['부도'] == 1) & (train['소속코드'] == 4)]  # 얘네들은 공시를 찾을 수 없거나 다른 사유에 의해서 부도

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자기자본배당률,금융비용 대 부채비율,자기자본배율,유동부채비율,...,유동자산회전률,총자본투자효율,순운전자본비율,누적수익성비율,총자산영업이익률,총자산회전율,년,key,공시제목,부도
89,(주)강원랜드,35250.0,2000/12,4.0,2003/09/04,UQ,4.97,0.000000,461.26,47.46,...,0.97,55.64,36.96,0.111201,0.130214,0.265266,2001.0,352502001,0,1.0
90,(주)강원랜드,35250.0,2001/12,4.0,2003/09/04,UQ,4.97,0.003868,461.26,47.46,...,0.97,55.64,36.96,0.335832,0.435723,0.664572,2002.0,352502002,0,1.0
91,(주)강원랜드,35250.0,2002/12,4.0,2003/09/04,UQ,6.94,0.002562,638.00,28.26,...,1.13,43.26,22.25,0.498198,0.362327,0.578167,2003.0,352502003,0,1.0
109,(주)경윤하이드로에너지,19120.0,2000/12,4.0,2011/04/06,UQ,3.49,0.064542,500.94,54.92,...,2.14,27.08,38.11,0.477951,0.074341,1.596995,2001.0,191202001,0,1.0
110,(주)경윤하이드로에너지,19120.0,2001/12,4.0,2011/04/06,UQ,3.07,0.041816,548.78,51.69,...,2.15,28.52,42.51,0.499863,0.085689,1.452628,2002.0,191202002,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27296,후야인포넷(주),32050.0,2000/12,4.0,2005/04/26,UQ,0.00,0.032603,654.91,37.11,...,1.85,13.97,49.69,0.081179,0.022520,1.017036,2001.0,320502001,0,1.0
27297,후야인포넷(주),32050.0,2001/12,4.0,2005/04/26,UQ,0.00,0.021178,360.12,33.95,...,1.24,-38.25,40.44,-0.444249,-0.191165,1.288789,2002.0,320502002,0,1.0
27298,후야인포넷(주),32050.0,2002/12,4.0,2005/04/26,UQ,0.00,0.021223,114.67,190.59,...,1.38,-42.34,11.37,0.000000,-0.368169,0.968728,2003.0,320502003,0,1.0
27299,후야인포넷(주),32050.0,2003/12,4.0,2005/04/26,UQ,0.00,0.206503,10.75,190.14,...,0.35,-252.34,-2.14,-3.185847,-1.857084,0.996953,2004.0,320502004,0,1.0


In [8]:
arr1 = dataset['거래소코드'].unique()
lis1 = arr1.tolist()

In [15]:
droplist = ['5호','인프라','네트','C&C','솔믹스','E&M','Metal']
for i in droplist:
    lis1.remove(i)

In [16]:
lis1

['030970',
 '064910',
 '022220',
 '049790',
 '033040',
 '042470',
 '027740',
 '017210',
 '021740',
 '036530',
 '040780',
 '024080',
 '033530',
 '020490',
 '060580',
 '063680',
 '021930',
 '030610',
 '058910',
 '022780',
 '034300',
 '037740',
 '017370',
 '066030',
 '035030',
 '044570',
 '053530',
 '052520',
 '032600',
 '027560',
 '024720',
 '052530',
 '013630',
 '033070',
 '039500',
 '051960',
 '051070',
 '050350',
 '049850',
 '023780',
 '049990',
 '048660',
 '035930',
 '021240',
 '015150',
 '047890',
 '022870',
 '025080',
 '046820',
 '044740',
 '045870',
 '046520',
 '045790',
 '045800',
 '045860',
 '044890',
 '045080',
 '030700',
 '044400',
 '044240',
 '044750',
 '044410',
 '043380',
 '043350',
 '043400',
 '017090',
 '033300',
 '045090',
 '043040',
 '043700',
 '042460',
 '042310',
 '042300',
 '040950',
 '040940',
 '040860',
 '039540',
 '040260',
 '040270',
 '040380',
 '039220',
 '039210',
 '040520',
 '039520',
 '038760',
 '039720',
 '038580',
 '038940',
 '039380',
 '039160',
 '039920',

In [21]:
textbox = []
for i in range(len(dataset)):
    html = dataset['text'][i]
    soup = BeautifulSoup(html, 'html.parser').get_text()
    
    if len(html[soup.find('5.'):soup.find('6.')]) > 1:
        textbox.append(html[soup.find('5.'):soup.find('6.')])
    else:
        textbox.append(html[soup.find('2.'): soup.find('3.')])


In [22]:
textbox2 = []
for i in range(len(textbox)):
    a = textbox[i].split('\n')
    try:
        textbox2.append(a[1])
    except:
        textbox2.append('')

In [23]:
dataset['폐지사유'] = textbox
dataset['폐지사유요약'] = textbox2
dataset

,회사명,거래소코드,공시발생일,공시제목,text,link,폐지사유,폐지사유요약
0,주은리스,030970,2002-11-28 18:54,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/28/K00...,5.등록취소사유\n액면가액일정비율 미달로 연속하여 10일간 지속\n\n\n,액면가액일정비율 미달로 연속하여 10일간 지속
1,글로벌파워,064910,2002-11-26 18:53,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/26/K00...,5.등록취소사유\n증권투자회사법 제64조의 규정에 의한 해산사유의 발생\n\n\n,증권투자회사법 제64조의 규정에 의한 해산사유의 발생
2,주은리스,030970,2002-11-21 18:24,상장폐지승인,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/21/K00...,5.등록취소사유\n액면가액일정비율 미달로 연속하여 10일간 지속\n\n\n,액면가액일정비율 미달로 연속하여 10일간 지속
3,정산애강,022220,2002-11-13 16:59,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/13/K00...,5.등록취소사유\n최종부도 및 은행거래정지\n\n\n,최종부도 및 은행거래정지
4,소프트윈,049790,2002-11-06 18:09,상장폐지((주)소프트윈),\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/06/K00...,5.등록취소사유\n최종부도 및 은행거래정지\n\n\n,최종부도 및 은행거래정지
...,...,...,...,...,...,...,...,...
933,유안타제2호스팩,219960,2018-02-08 17:51,상장폐지,\n\n\n\n\n\n\r\n\t.PGBRK\t\t\t{page-break-afte...,https://kind.krx.co.kr/external/2018/02/08/001...,5.상장폐지사유\n상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사...,상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소
934,셀트리온,068270,2018-02-07 15:48,상장폐지(유가증권시장 상장),\n\n\n\n\n\n\r\n\t.PGBRK\t\t\t{page-break-afte...,https://kind.krx.co.kr/external/2018/02/07/000...,5.상장폐지사유\n유가증권시장 상장\n\n\n,유가증권시장 상장
935,키움스팩3호,218710,2018-02-01 18:08,상장폐지,\n\n\n\n\n\n\r\n\t.PGBRK\t\t\t{page-break-afte...,https://kind.krx.co.kr/external/2018/02/01/000...,5.상장폐지사유\n상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사...,상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소
936,골든브릿지제3호스팩,219580,2018-01-25 18:10,상장폐지,\n\n\n\n\n\n\r\n\t.PGBRK\t\t\t{page-break-afte...,https://kind.krx.co.kr/external/2018/01/25/000...,5.상장폐지사유\n상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사...,상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소


In [32]:
(dataset.공시발생일.str[0:4].astype(int) -1).astype(str)

0      2001
1      2001
2      2001
3      2001
4      2001
       ... 
933    2017
934    2017
935    2017
936    2017
937    2017
Name: 공시발생일, Length: 938, dtype: object

In [33]:
dataset['key'] = dataset['거래소코드'] + (dataset.공시발생일.str[0:4].astype(int) -1).astype(str)
dataset.head()

,회사명,거래소코드,공시발생일,공시제목,text,link,폐지사유,폐지사유요약,key
0,주은리스,030970,2002-11-28 18:54,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/28/K00...,5.등록취소사유\n액면가액일정비율 미달로 연속하여 10일간 지속\n\n\n,액면가액일정비율 미달로 연속하여 10일간 지속,0309702001
1,글로벌파워,064910,2002-11-26 18:53,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/26/K00...,5.등록취소사유\n증권투자회사법 제64조의 규정에 의한 해산사유의 발생\n\n\n,증권투자회사법 제64조의 규정에 의한 해산사유의 발생,0649102001
2,주은리스,030970,2002-11-21 18:24,상장폐지승인,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/21/K00...,5.등록취소사유\n액면가액일정비율 미달로 연속하여 10일간 지속\n\n\n,액면가액일정비율 미달로 연속하여 10일간 지속,0309702001
3,정산애강,022220,2002-11-13 16:59,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/13/K00...,5.등록취소사유\n최종부도 및 은행거래정지\n\n\n,최종부도 및 은행거래정지,0222202001
4,소프트윈,049790,2002-11-06 18:09,상장폐지((주)소프트윈),\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/06/K00...,5.등록취소사유\n최종부도 및 은행거래정지\n\n\n,최종부도 및 은행거래정지,0497902001


In [34]:
dataset = dataset.fillna(0)

In [35]:
train['key'] = train['key'].astype(str)

In [36]:
train = train.merge(dataset[['공시발생일','공시제목','text','폐지사유','폐지사유요약','key']], on='key', how= 'outer')
train.shape

(28257, 23)

In [37]:
train[['공시발생일','공시제목','text','폐지사유','폐지사유요약']] = train[['공시발생일','공시제목','text','폐지사유','폐지사유요약']].fillna(0)

In [38]:
train = train.dropna()
train.shape

(27225, 23)

In [39]:
train.to_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\최종데이터셋\Finaldataset.csv', index = False)

# **부도 사유 상세 정리**

In [40]:
dataset = pd.read_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\최종데이터셋\Finaldataset.csv')

In [44]:
dataset.loc[dataset['부도'] == 0]['공시제목'].value_counts()

0                                         435
상장폐지                                        4
[정정]상장폐지                                    4
[정정]상장폐지(상장폐지무효확인 청구의 소(원고 상고 기각))          1
[정정]상장폐지(상장폐지결정 등 효력정지 가처분 결정 취소(대법원))      1
Name: 공시제목, dtype: int64

In [16]:
pdataset = pd.read_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\Dataset.csv', index_col=0)
pdataset

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계_요약_,유동자산_요약_,당좌자산_요약_,현금_및_현금성자산_요약_,...,기업가치_EV_,EBITDA,EBITDA_매출액,EBITDA_금융비용,EBITDA_평균발행주식수,EV_EBITDA,년,key,공시제목,부도
0,(주)CMG제약,58820.0,2000/12,5.0,0,UQ,28990.0,21169.0,10541.0,4330.0,...,69545.33,-1288.45,-71.74,0.00,-104.44,0.00,2001.0,588202001,0,1.0
1,(주)CMG제약,58820.0,2001/12,5.0,0,UQ,20908.0,16359.0,2307.0,1829.0,...,69545.33,-1288.45,-71.74,0.00,-104.44,0.00,2002.0,588202002,0,1.0
2,(주)CMG제약,58820.0,2002/12,5.0,0,UQ,15401.0,9880.0,1436.0,355.0,...,69545.33,-1288.45,-71.74,0.00,-104.44,0.00,2003.0,588202003,0,1.0
3,(주)CMG제약,58820.0,2003/12,5.0,0,UQ,11063.0,8534.0,1083.0,2.0,...,69545.33,-1288.45,-71.74,0.00,-104.44,0.00,2004.0,588202004,0,1.0
4,(주)CMG제약,58820.0,2004/12,5.0,0,UQ,5179.0,3893.0,445.0,11.0,...,69545.33,-1288.45,-71.74,0.00,-104.44,0.00,2005.0,588202005,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,흥구석유(주),24060.0,2016/12,5.0,0,UQ,79137.0,14794.0,12274.0,1440.0,...,55934.79,1348.52,0.94,10.71,89.90,41.48,2017.0,240602017,0,1.0
27353,흥구석유(주),24060.0,2017/12,5.0,0,UQ,79912.0,15880.0,13124.0,1588.0,...,46036.63,1147.56,0.74,9.02,76.50,40.12,2018.0,240602018,0,1.0
27354,흥구석유(주),24060.0,2018/12,5.0,0,UQ,75206.0,12515.0,11029.0,4017.0,...,36557.57,1451.79,0.87,16.83,96.79,25.18,2019.0,240602019,0,1.0
27355,흥구석유(주),24060.0,2019/12,5.0,0,UQ,83661.0,22620.0,19701.0,2318.0,...,78081.78,1009.25,0.67,507.93,67.28,77.37,2020.0,240602020,0,1.0


In [17]:
pdataset.loc[pdataset['회사명'] == '(주)국순당']

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계_요약_,유동자산_요약_,당좌자산_요약_,현금_및_현금성자산_요약_,...,기업가치_EV_,EBITDA,EBITDA_매출액,EBITDA_금융비용,EBITDA_평균발행주식수,EV_EBITDA,년,key,공시제목,부도
299,(주)국순당,43650.0,2000/12,5.0,0,UQ,60058.0,43765.0,41069.0,26339.0,...,68565.70,10322.64,16.68,0.0,578.03,6.64,2001.0,436502001,[정정]내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,1.0
300,(주)국순당,43650.0,2001/12,5.0,0,UQ,99133.0,76403.0,73795.0,63114.0,...,68565.70,10322.64,16.68,0.0,578.03,6.64,2002.0,436502002,[정정]내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,1.0
301,(주)국순당,43650.0,2002/12,5.0,0,UQ,124382.0,98703.0,96561.0,62967.0,...,68565.70,10322.64,16.68,0.0,578.03,6.64,2003.0,436502003,[정정]내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,1.0
302,(주)국순당,43650.0,2003/12,5.0,0,UQ,148460.0,106619.0,102647.0,44257.0,...,68565.70,10322.64,16.68,0.0,578.03,6.64,2004.0,436502004,[정정]내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,1.0
303,(주)국순당,43650.0,2004/12,5.0,0,UQ,157614.0,95137.0,90668.0,39563.0,...,68565.70,10322.64,16.68,0.0,578.03,6.64,2005.0,436502005,[정정]내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,1.0
304,(주)국순당,43650.0,2005/12,5.0,0,UQ,169999.0,103748.0,98904.0,39407.0,...,68565.70,10322.64,16.68,0.0,578.03,6.64,2006.0,436502006,[정정]내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,1.0
305,(주)국순당,43650.0,2006/12,5.0,0,UQ,157086.0,82942.0,77847.0,32684.0,...,68565.70,10322.64,16.68,0.0,578.03,6.64,2007.0,436502007,[정정]내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,1.0
306,(주)국순당,43650.0,2007/12,5.0,0,UQ,158359.0,71869.0,66983.0,28762.0,...,68565.70,10322.64,16.68,0.0,578.03,6.64,2008.0,436502008,[정정]내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,1.0
307,(주)국순당,43650.0,2008/12,5.0,0,UQ,148528.0,72792.0,63571.0,24626.0,...,-24625.80,7761.16,14.35,0.0,434.60,-3.17,2009.0,436502009,[정정]내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,1.0
308,(주)국순당,43650.0,2009/12,5.0,0,UQ,153877.0,69966.0,62787.0,48137.0,...,121517.34,9848.03,17.96,0.0,551.45,12.34,2010.0,436502010,[정정]내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,1.0


In [19]:
dataset.loc[dataset['부도'] == 0]['회사명'].unique()

array(['(주)나노트로닉스', '(주)나노하이텍', '(주)나리지*온', '(주)네오퍼플', '(주)넥스지',
       '(주)넷컴스토리지', '(주)뉴젠아이씨티', '(주)단성일렉트론', '(주)대국', '(주)대양글로벌',
       '(주)데코앤이', '(주)도움', '(주)동방라이텍', '(주)동서정보기술', '(주)두림티앤씨',
       '(주)드림티엔터테인먼트', '(주)디보스', '(주)디아이디', '(주)디에스', '(주)디에스피이엔티',
       '(주)디지텍시스템스', '(주)디패션', '(주)레이더스컴퍼니', '(주)렉스엘이앤지', '(주)룩손에너지홀딩스',
       '(주)리더컴', '(주)마이스코', '(주)맥스브로', '(주)맥시스템', '(주)모다', '(주)모디아',
       '(주)모린스', '(주)미디어코프', '(주)미성포리테크', '(주)바이오빌', '(주)바이오시스',
       '(주)보광티에스', '(주)보성인터내셔날', '(주)보홍', '(주)블루스톤디앤아이', '(주)블루젬디앤씨',
       '(주)비앤비성원', '(주)비엔씨컴퍼니', '(주)비이티', '(주)사이노젠', '(주)삼우이엠씨',
       '(주)샤인시스템', '(주)성진산업', '(주)세니콘', '(주)세라온홀딩스', '(주)세븐코스프', '(주)세실',
       '(주)세화피앤씨', '(주)소예', '(주)솔빛미디어', '(주)슈마일렉트론', '(주)스탠더드텔레콤',
       '(주)스톰이앤에프', '(주)스틸플라워', '(주)스페이스솔루션', '(주)시냅스엠', '(주)시스컴',
       '(주)시큐어소프트', '(주)신양오라컴디스플레이', '(주)신지소프트', '(주)신한에스아이티', '(주)쌈지',
       '(주)써미트테크놀로지', '(주)썬코어', '(주)썬테크놀로지스', '(주)쏠라엔텍', '(주)쓰리소프트',
       '(주)씨그널엔터테인먼트그룹', '(주)씨모스', '(주)씨모텍', 